In [ ]:
!pip install newsapi-python

In [ ]:
!pip install yfinance

In [ ]:
!pip install vaderSentiment

In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
from newsapi import NewsApiClient
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from scipy.optimize import minimize

In [ ]:
# Define the tickers of the assets to include in the portfolio
tickers = ['AAPL', 'GOOG', 'AMZN', 'META', 'TSLA']

In [ ]:
NEWS_API_KEY = '08f2c33f0f9b4dfa9a907afc725bb0c7'
# Initialize the News API client and the VADER sentiment analyzer
newsapi = NewsApiClient(api_key= NEWS_API_KEY)
analyzer = SentimentIntensityAnalyzer()

In [ ]:
# Define the date range for the news articles
date_from = '2023-03-30'
date_to = '2023-04-30'

In [ ]:
prices = yf.download(tickers, period='max')['Adj Close']

In [ ]:
# Define a function to fetch the news articles and calculate their sentiment scores
def get_sentiment_scores(keyword, date_from, date_to):
    articles = newsapi.get_everything(q=keyword, from_param=date_from, to=date_to, language='en')
    scores = [analyzer.polarity_scores(article['title'])['compound'] for article in articles['articles']]
    return scores

In [ ]:
# Fetch the sentiment scores for each stock
sentiment_scores = {}
for ticker in tickers:
    keyword = ticker + ' stock'
    scores = get_sentiment_scores(keyword, date_from, date_to)
    sentiment_scores[ticker] = np.mean(scores)


In [ ]:
# Calculate the expected returns and covariance matrix of the assets
returns = np.log(prices / prices.shift(1)).mean()
cov_matrix = np.log(prices / prices.shift(1)).cov()

In [ ]:
# Define the objective function to minimize
def neg_sharpe_ratio(weights, returns, cov_matrix, sentiment_scores):
#    print('weights:', weights)
#    print('returns:', returns.values)
#    print('sentiment_scores:', list(sentiment_scores.values))
#    print('sentiment_scores:', list(sentiment_scores.values()))
    portfolio_return = np.dot(weights, returns.values) + np.dot(weights, list(sentiment_scores.values()))
    portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    sharpe_ratio = portfolio_return / portfolio_volatility
    return -sharpe_ratio

In [ ]:
def neg_sharpe_ratio_wosenti(weights, returns):
    port_returns = np.sum(returns.mean() * weights) * 252
    port_volatility = np.sqrt(np.dot(weights.T, np.dot(cov_matrix * 252, weights)))
    return -port_returns / port_volatility

In [ ]:
# Define the constraints of the optimization problem
constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})

In [ ]:
# Define the bounds of the optimization problem
bounds = tuple((0, 1) for _ in range(len(tickers)))

In [ ]:
# Solve the optimization problem to find the optimal portfolio weights
initial_weights = [1 / len(tickers) for _ in range(len(tickers))]
opt_result = minimize(neg_sharpe_ratio, initial_weights, args=(returns, cov_matrix, sentiment_scores), method='SLSQP', bounds=bounds, constraints=constraints)


In [ ]:
print('opt_result:', np.round(opt_result.x*100,1), '%')
opt_weights = opt_result.x
print(np.round(-opt_result.fun,2))

In [ ]:
# Print the optimal portfolio weights and the maximum Sharpe ratio
print('Optimal portfolio weights:', opt_weights)
print('Maximum Sharpe ratio:', -opt_result.fun)

In [ ]:
#without sentiments
opt_result_without_sentiment = minimize(neg_sharpe_ratio_wosenti, initial_weights, args=returns, method='SLSQP', bounds=bounds, constraints=constraints)



In [ ]:
print('opt_result_without_sentiment:', np.round(opt_result_without_sentiment.x*100,2), '%')
opt_weightswo_sentiment = opt_result_without_sentiment.x
print(np.round(-opt_result_without_sentiment.fun,2))